In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Filtter wfs with bounding box

This notebook filters a WFS request with a bounding box. The building WFS service of pdok is used, a.k.a. the [bag](https://www.pdok.nl/geo-services/-/article/basisregistratie-adressen-en-gebouwen-ba-1#45d074fd320dabee548d5a8bf1fed85e).

In [2]:
import logging
from itertools import chain
from pathlib import Path

import geopandas as gpd
import ogr
from owslib.etree import etree
from owslib.wfs import WebFeatureService

from building_detector import helpers


logging.basicConfig(
    format='%(message)s',
    level=logging.INFO
)
wfs_url = 'https://geodata.nationaalgeoregister.nl/bag/wfs/v1_1'

In [3]:
wfs = WebFeatureService(wfs_url, version='2.0.0')
wfs.identification.title

'BAG WFS'

In [4]:
print('The WFS has the following layers')
list(wfs.contents)

The WFS has the following layers


['bag:bag',
 'bag:ligplaats',
 'bag:pand',
 'bag:standplaats',
 'bag:verblijfsobject',
 'bag:woonplaats']

We need the `'bag:pand'` (`'pand'` is `'building'` in Dutch).

In [5]:
layer_name = 'bag:pand'
wfs.get_schema(layer_name)

{'properties': {'gid': 'long',
  'identificatie': 'string',
  'bouwjaar': 'integer',
  'status': 'string',
  'gebruiksdoel': 'string',
  'oppervlakte_min': 'integer',
  'oppervlakte_max': 'integer',
  'aantal_verblijfsobjecten': 'integer',
  'rdf_seealso': 'string'},
 'geometry': 'GeometryCollection',
 'geometry_column': 'geometrie'}

Get the bounding box of some gml:

In [6]:
muni = 'Amsterdam'
muni_path = Path(f'../data/{muni}.gml'.lower())
envelope = gpd.read_file(muni_path).envelope
bbox = helpers.Bbox.from_poly(envelope[0])
bbox

BBox(xmin=110188.384, ymin=476770.797, xmax=134029.81, ymax=493893.636)

In [7]:
response = wfs.getfeature(
    typename=layer_name,
    bbox=tuple(chain(*bbox)),
    maxfeatures=1000,            # combined can be
    startindex=2000              # used to do paging
)

layer_name_clean = layer_name.replace(':', '_')
layer_path = muni_path.parent / f'{muni_path.stem}_{layer_name_clean}.gml'
layer_path.write_bytes(response.read())

1534256